In [1]:
import numpy as np
import tensorflow as tf
tf.__version__

/Users/georgymh/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.6.0'

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
X_train = np.concatenate((mnist.train.images, mnist.validation.images))
y_train = np.concatenate((
            np.asarray(mnist.train.labels, dtype=np.int32),
            np.asarray(mnist.validation.labels, dtype=np.int32),
          ))
X_test = mnist.test.images
y_test = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [3]:
X_train_ = X_train.reshape(-1, 784)
y_train_ = np.eye(10)[y_train]
X_test_ = X_test.reshape(-1, 784)
y_test_ = np.eye(10)[y_test]
X_train_.shape, y_train_.shape

((60000, 784), (60000, 10))

In [96]:
class Perceptron:
    def __init__(self):
        self.n_input = 784
        self.n_hidden1 = 200
        self.n_hidden2 = 200
        self.n_classes = 10

    def preprocess_input(self, x):
        return x

    def build_model(self, input_layer):
        #['layer1/dense/kernel:0', 'layer1/dense/bias:0', 'layer2/dense/kernel:0',
        #'layer2/dense/bias:0', 'logits_layer/dense/kernel:0', 'logits_layer/dense/bias:0']
        new_weights = self.new_weights if hasattr(self, 'new_weights') else None
        layer_name = "layer1"
        with tf.variable_scope(layer_name):
            kernel_init, bias_init = None, None
            if new_weights:
                kernel_init = tf.constant_initializer(new_weights['/'.join([layer_name, "dense", "kernel"]) + ":0"])
                bias_init = tf.constant_initializer(new_weights['/'.join([layer_name, "dense", "bias"]) + ":0"])
            self.layer1 = tf.layers.dense(input_layer, self.n_hidden1, tf.nn.relu,
                kernel_initializer=kernel_init, bias_initializer=bias_init)
        layer_name = "layer2"
        with tf.variable_scope(layer_name):
            kernel_init, bias_init = None, None
            if new_weights:
                kernel_init = tf.constant_initializer(new_weights['/'.join([layer_name, "dense", "kernel"]) + ":0"])
                bias_init = tf.constant_initializer(new_weights['/'.join([layer_name, "dense", "bias"]) + ":0"])
            self.layer2 = tf.layers.dense(self.layer1, self.n_hidden2, tf.nn.relu,
                kernel_initializer=kernel_init, bias_initializer=bias_init)
        layer_name = "logits_layer"
        with tf.variable_scope(layer_name):
            kernel_init, bias_init = None, None
            if new_weights:
                kernel_init = tf.constant_initializer(new_weights['/'.join([layer_name, "dense", "kernel"]) + ":0"])
                bias_init = tf.constant_initializer(new_weights['/'.join([layer_name, "dense", "bias"]) + ":0"])
            self.logits = tf.layers.dense(self.layer2, self.n_classes,
                kernel_initializer=kernel_init, bias_initializer=bias_init)

    def build_loss(self):
        self.loss = tf.losses.sparse_softmax_cross_entropy(
            labels=self.labels,
            logits=self.logits,
            loss_collection=tf.GraphKeys.LOSSES
        )

    def build_optimizer(self):
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        self.optimizer = optimizer.minimize(
            loss=self.loss,
            global_step=tf.train.get_global_step(),
            name='train_op'
        )

    def build_predictions_obj(self):
        logits = self.logits
        classes = tf.argmax(input=self.logits, axis=1, name="classes_tensor")
        probabilities = tf.nn.softmax(self.logits, name="softmax_tensor")

        tf.add_to_collection('predictions', logits)
        tf.add_to_collection('predictions', classes)
        tf.add_to_collection('predictions', probabilities)

        self.predictions = {
            "logits": logits,
            "classes": classes,
            "probabilities": probabilities
        }

    def build_eval_metric(self):
        self.eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(labels=self.labels, predictions=self.predictions["classes"])
        }

    def get_estimator(self, mode):
        estimator = None
        self.build_predictions_obj()
        if mode == tf.estimator.ModeKeys.PREDICT:
            self.build_eval_metric()
            estimator = tf.estimator.EstimatorSpec(mode=mode, predictions=self.predictions)
        elif mode == tf.estimator.ModeKeys.TRAIN:
            self.build_loss()
            self.build_optimizer()
            estimator = tf.estimator.EstimatorSpec(mode=mode, loss=self.loss, train_op=self.optimizer)
        elif mode == tf.estimator.ModeKeys.EVAL:
            self.build_loss()
            self.build_eval_metric()
            estimator = tf.estimator.EstimatorSpec(mode=mode, loss=self.loss, eval_metric_ops=self.eval_metric_ops)
        return estimator


    def get_model(self, features, labels, mode, params):
        """
        When using the Estimator API, features will come as a TF Tensor already.
        """
        # Set up hyperparameters.
        if params:
            self.learning_rate = params.get("learning_rate", None)
            self.new_weights = params.get("new_weights", None)

        # Do pre-processing if necessary.
        self.input_layer = self.preprocess_input(features["x"])
        self.labels = labels


        # def init_fn(scaffold, sess):
        #     tf.set_random_seed(0)
        #     sess.run(tf.global_variables_initializer())
        # scaffold = tf.train.Scaffold(init_fn=init_fn)

        # Define the model.
        self.build_model(self.input_layer)
        
        #####
#         with tf.Session().as_default() as sess:
#             sess.run(tf.global_variables_initializer())
#             collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
#             weights = {tensor.name:sess.run(tensor) for tensor in collection}
#         print(weights['layer1/dense/bias:0'][:5])
        #####
        
        # Build and return the estimator.
        return self.get_estimator(mode)

    def load_weights(self, new_weights, metagraph_file, checkpoint_dir):
        tf.reset_default_graph()
        with tf.Session().as_default() as sess:
            new_saver = tf.train.import_meta_graph(metagraph_file)
            # To load non-trainable variables and prevent errors...
            # we restore them if they are found, or initialize them otherwise.
            try:
                new_saver.restore(sess, tf.train.latest_checkpoint(checkpoint_dir))
            except:
                sess.run(tf.global_variables_initializer())

            collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            for tensor in collection:
                assign_op = tensor.assign(new_weights[tensor.name])
                sess.run(assign_op)

            save_path = new_saver.save(sess, checkpoint_dir + "model.ckpt")

    def get_weights(self, latest_checkpoint):
        tf.reset_default_graph()
        graph = tf.Graph()
        with tf.Session(graph=graph) as sess:
            new_saver = tf.train.import_meta_graph(latest_checkpoint + '.meta')
            new_saver.restore(sess, latest_checkpoint)
            collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            weights = {tensor.name:sess.run(tensor) for tensor in collection}
        return weights

    def sum_weights(self, weights1, weights2):
        new_weights = {}
        for key1, key2 in zip(sorted(weights1.keys()), sorted(weights2.keys())):
            assert key1 == key2, 'Error with keys'
            new_weights[key1] = weights1[key1] + weights2[key2]
        return new_weights

    def scale_weights(self, weights, factor):
        new_weights = {}
        for key, value in weights.items():
            new_weights[key] = value * factor
        return new_weights

    def inverse_scale_weights(self, weights, factor):
        new_weights = {}
        for key, value in weights.items():
            new_weights[key] = value / factor
        return new_weights


In [76]:
# # Create the Estimator
# model = Perceptron()
# classifier = tf.estimator.Estimator(model_fn=model.get_model, model_dir="./mnist/")

# # Set up logging for predictions
# tensors_to_log = {"probabilities": "softmax_tensor"}
# logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=1)

In [73]:
"""
ESTIMATOR WAY
1- Server sends its 'input_fn()' defining the model and initial weights to clients 
    [yes]
2- Clients create an estimator with the weights, train on their data, and send back the new weights to the server 
    [yes -- put weights in folder, create estimator with checkpoint, run fit on estimator, sends the latest checkpoint]
3- As the server receives the new weights, it averages the weights; once done, it evaluates the model on a test set 
    [yes -- as weights are received we average them; once done, we load the model, assign the weights, save the model,
    under new name, and run an eval on the test set]


TRADITIONAL WAY

"""

tf.reset_default_graph()

# CLIENT
def train_model(config):
    """
    Loads the model and weights found in server-checkpoints/ (if found), trains using the hyperparameters 
    in config, then returns a list of new checkpoints after training. 
    """
    def make_weights_dict(sess, collection):
        return {tensor.name:sess.run(tensor) for tensor in collection}
    
    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:
        #new_saver = tf.train.import_meta_graph(metagraph)
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        try:
            new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/'))
        except e:
            # Couldn't load model.
            print("Error...")
            raise e 
        
        input = graph.get_collection('input_tensor:0')
        print(input)
        
        logits, classes, probabilities = graph.get_collection('predictions')
        print(logits)
        
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        print(make_weights_dict(sess, collection))
        
        for i in range(20000):
            batch = mnist.train.next_batch(50)
            train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))
            if i % 1000 == 0:
                # test accuracy
            

        print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

# SERVER
def aggregate_models(config):
    """
    Loads the models found in updates/
    """
    

#train_model(None)

IndentationError: expected an indented block (<ipython-input-73-6b57660b5e1e>, line 57)

In [56]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
#chkp.print_tensors_in_checkpoint_file(tf.train.latest_checkpoint('./mnist/'), tensor_name='', all_tensors=True)

In [109]:
def get_initial_weights():
    tf.reset_default_graph()
    m = Perceptron()
    initial = tf.placeholder(tf.float32, shape=(None, 28*28), name="input_tensor")
    _ = m.get_model(features={"x": initial}, labels=None, mode='predict', params=None)
    with tf.Session().as_default() as sess:
        sess.run(tf.global_variables_initializer())
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        weights = {tensor.name:sess.run(tensor) for tensor in collection}
    tf.reset_default_graph()
    return weights

initial_weights = get_initial_weights()
print(initial_weights['layer1/dense/bias:0'][:5])

[0.07752929 0.0894733  0.05860019 0.10548263 0.00462598]


In [111]:
%%time
# Create the Estimator
params = {'learning_rate': 0.001, 'new_weights': initial_weights}
model = Perceptron()
classifier1 = tf.estimator.Estimator(
    model_fn=model.get_model, 
    model_dir="./mnist1/",
    params=params
)


# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_},
    y=y_train,
    batch_size=1,
    num_epochs=1,
    shuffle=False
)
classifier1.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist1/lol/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11cf447b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist1/lol/model.ckpt.
INFO:tensorflow:loss = 2.2414126, step = 1
INFO:tensorflow:global

In [102]:
%%time
# Create the Estimator
params = {'learning_rate': 0.01, 'new_weights': initial_weights}
model = Perceptron()
classifier2 = tf.estimator.Estimator(
    model_fn=model.get_model, 
    model_dir="./mnist2/",
    params=params
)


# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_},
    y=y_train,
    batch_size=1,
    num_epochs=None,
    shuffle=False
)
classifier2.train(input_fn=train_input_fn, steps=100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11cf44e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist2/model.ckpt.
INFO:tensorflow:loss = 1.6577064, step = 1
INFO:tensorflow:Saving checkpo

In [103]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_},
    y=y_test,
    num_epochs=1,
    shuffle=False
)
eval_results = classifier1.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-01-00:18:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist1/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-01-00:18:37
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.5769, global_step = 100, loss = 1.3338704
{'accuracy': 0.5769, 'loss': 1.3338704, 'global_step': 100}


In [104]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_},
    y=y_test,
    num_epochs=1,
    shuffle=False
)
eval_results = classifier2.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-01-00:18:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist2/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-01-00:18:39
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.3963, global_step = 100, loss = 1.7706457
{'accuracy': 0.3963, 'loss': 1.7706457, 'global_step': 100}


In [106]:
def import_weights_as_np(path):
    tf.reset_default_graph()
    weights = []
    with tf.Session().as_default() as sess:
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint(path) + '.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint(path))
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        weights = {tensor.name:sess.run(tensor) for tensor in collection}
    return weights
        
weights1 = import_weights_as_np('./mnist1/')
weights2 = import_weights_as_np('./mnist2/')

print(sum(w.size for _, w in weights1.items()), weights1['layer1/dense/bias:0'][:5])
print(sum(w.size for _, w in weights2.items()), weights2['layer1/dense/bias:0'][:5])
print([k for k, _ in weights2.items()])

INFO:tensorflow:Restoring parameters from ./mnist1/model.ckpt-100
INFO:tensorflow:Restoring parameters from ./mnist2/model.ckpt-100
199210 [-0.00913986  0.10991648  0.04031864  0.09077774 -0.10459123]
199210 [-0.06314074  0.06910972 -0.00736872  0.05201993 -0.15259027]
['layer1/dense/kernel:0', 'layer1/dense/bias:0', 'layer2/dense/kernel:0', 'layer2/dense/bias:0', 'logits_layer/dense/kernel:0', 'logits_layer/dense/bias:0']


In [107]:
def sum_weights(weights1, weights2):
    new_weights = {}
    for key1, key2 in zip(sorted(weights1.keys()), sorted(weights2.keys())):
        assert key1 == key2, 'Error with keys'
        new_weights[key1] = (weights1[key1] + weights2[key2])/2
    return new_weights

new_weights = sum_weights(weights1, weights2)
print(sum(w.size for _, w in new_weights.items()), new_weights['layer1/dense/bias:0'][:5])

199210 [-0.0361403   0.0895131   0.01647496  0.07139883 -0.12859075]


In [108]:
%%time
# Create the Estimator
model = Perceptron()
params = {'learning_rate': 0.01, 'new_weights': new_weights}
classifier3 = tf.estimator.Estimator(
    model_fn=model.get_model, 
    model_dir="./mnist3/",
    params=params
)


# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train_},
    y=y_train,
    batch_size=1,
    num_epochs=None,
    shuffle=False
)

classifier3.train(input_fn=train_input_fn, steps=100)


# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test_},
    y=y_test,
    num_epochs=1,
    shuffle=False
)
eval_results = classifier3.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist3/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11d679630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./mnist3/model.ckpt.
INFO:tensorflow:loss = 0.757868, step = 1
INFO:tensorflow:Saving checkpoi

In [58]:
def load_weights(weights, round_num):
    tf.reset_default_graph()
    with tf.Session().as_default() as sess:
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/')) # to load non-trainable variables 
        
        collection = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        for tensor in collection:
            assign_op = tensor.assign(weights[tensor.name])
            sess.run(assign_op)
        
        save_path = new_saver.save(sess, "./mnist/model.ckpt", global_step=round_num)
        print("Model saved in path: %s" % save_path)
load_weights(new_weights, 1337)

INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt
Model saved in path: ./mnist/model.ckpt-1337


In [70]:
from google.protobuf import json_format

def export_metagraph_as_json(model, hparams):
    tf.reset_default_graph()
    with tf.Graph().as_default() as graph:
        m = model(hparams)
        m.build_model(tf.placeholder(tf.float32, (None, 28*28)))
        m.build_predictions_obj()
        graph_def = graph.as_graph_def()
        json_string = json_format.MessageToJson(graph_def)
    return json_string

export_metagraph_as_json(Perceptron, hparams)[:50]

'{\n  "node": [\n    {\n      "name": "Placeholder",\n '

In [75]:
def export_metagraph():
    tf.reset_default_graph()
    with tf.Session().as_default() as sess:
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
#         m = model(hparams)
#         m.build_model(tf.placeholder(tf.float32, (None, 28*28), name='input_tensor'))
#         m.build_predictions_obj()
        meta_graph_def = tf.train.export_meta_graph()
    return meta_graph_def

#mgraph = export_metagraph()
#metagraph = export_metagraph(Perceptron, hparams)
tf.train.latest_checkpoint('./mnist/')

'./mnist/model.ckpt-23613'

In [27]:
def build_latest_graph():
    tf.reset_default_graph()
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
        #new_saver = tf.train.import_meta_graph(metagraph)
        new_saver = tf.train.import_meta_graph(tf.train.latest_checkpoint('./mnist/') + '.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint('./mnist/'))
        
        print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
        
        input = loaded_graph.get_tensor_by_name('enqueue_input/Placeholder_1:0')
        #input = loaded_graph.get_collection('enqueue_input')
        #input = loaded_graph.get_collection('input_tensor:0')
        print(input)
        
        logits, classes, probabilities = loaded_graph.get_collection('predictions')
        print(logits)
        
#         def get_model(features, labels, mode):
#             return tf.estimator.EstimatorSpec(mode=mode, predictions=classes)
            
        
#         hparams = {'learning_rate': 0.001}
#         model = Perceptron(hparams)
#         classifier = tf.estimator.Estimator(model_fn=get_model, model_dir="mnist2/")
#         # Evaluate the model and print results
#         eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#             x={"x": X_test_},
#             y=y_test,
#             num_epochs=1,
#             shuffle=False
#         )
#         eval_results = classifier.evaluate(input_fn=eval_input_fn)
#         print(eval_results)
        
        
        
#         pred = sess.run(classes, feed_dict={input: X_test_[:10]})
#         print(pred, y_test[:10])
        
        #print(sess.run(predictions[0]))
    #print(sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)))
    #   for step in xrange(1000000):
    #     sess.run(train_op)

build_latest_graph()

INFO:tensorflow:Restoring parameters from ./mnist/model.ckpt-23613
[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>, <tf.Variable 'layer1/dense/kernel:0' shape=(784, 200) dtype=float32_ref>, <tf.Variable 'layer1/dense/bias:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'layer2/dense/kernel:0' shape=(200, 200) dtype=float32_ref>, <tf.Variable 'layer2/dense/bias:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'logits_layer/dense/kernel:0' shape=(200, 10) dtype=float32_ref>, <tf.Variable 'logits_layer/dense/bias:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>, <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>, <tf.Variable 'layer1/dense/kernel/Adam:0' shape=(784, 200) dtype=float32_ref>, <tf.Variable 'layer1/dense/kernel/Adam_1:0' shape=(784, 200) dtype=float32_ref>, <tf.Variable 'layer1/dense/bias/Adam:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'layer1/dense/bias/Adam_1:0' shape=(200,) dtype=float32_ref>, <tf.Variable 'la

In [ ]:
tf.reset_default_graph()

m = Perceptron(hparams)
m.build_model(tf.placeholder(tf.float32, (None, 28*28)))

weights = []
saver = tf.train.Saver()
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  #new_saver = tf.train.import_meta_graph('mnist/model.ckpt-23605.meta')
  #new_saver.restore(sess, tf.train.latest_checkpoint('mnist'))
  #saver.restore(sess, "./mnist/model.ckpt") 
    
  ckpt = tf.train.get_checkpoint_state("./mnist/model.ckpt-23604")
  print(ckpt)
  
    
  print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
  weights = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
weights

In [68]:
def prepare_for_training():
    tf.reset_default_graph()
    with tf.Session().as_default() as sess:
        m = Perceptron(hparams)
        m.build_all()
        tf.train.import_meta_graph(filename='./mnist/lol.meta')
    tf.reset_default_graph()

prepare_for_training()

TypeError: build_model() missing 1 required positional argument: 'input_layer'

In [82]:
tf.train.latest_checkpoint('./mnist')

'./mnist/model.ckpt-2'